In [11]:
%reload_ext autoreload
%autoreload 2
# %matplotlib widget
%matplotlib notebook

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
import os
import json
from tqdm import tqdm

In [13]:
# data_path = os.path.expanduser("~/Downloads/datasets/RS_v2_2010-10")
data_path = "/mnt/nas/datasets/reddit_dumps/RS_2019-08"

In [14]:
# Build a pandas dataframe with the data from the file. https://stackoverflow.com/a/56746204/3000741
# I want title, over_18, subreddit, and url

In [15]:
# will need domain to see if it's media or not. Let me check what I used in my original scraper.
# I just checked the mime type and file extension of the link. I might have to do that.
# 630298 entries. Let's see if we can load them all into memory!
cols = ["title", "over_18", "subreddit", 'url', "is_self", "domain", "score"]

In [16]:
def extract_keys(data, keys):
    out = {}
    for key in keys:
        out[key] = data.get(key)
    return out


In [17]:
filtered_data = []
with open(data_path, "r") as f:
    for line in tqdm(f, total=722000000):
        jl = json.loads(line)
        url = jl["url"]
        if (jl["is_self"] == True) or not url.endswith((".jpg", ".png")):
            continue
        line_data = extract_keys(json.loads(line), cols)
        filtered_data.append(line_data)

  3%|▎         | 21927461/722000000 [38:34<20:31:48, 9472.15it/s] 


In [94]:
df = pandas.DataFrame(filtered_data, columns=cols)

In [95]:
df

,title,over_18,subreddit,url,is_self,domain,score
0,Just when I thought I was getting good !!,False,gaming,http://imgur.com/nWBNd.jpg,False,imgur.com,29
1,Someone just sent this to me through my colleg...,False,atheism,https://www.reddit.com/r/atheism/comments/dl97...,True,self.atheism,7
2,I just opened a fortune cookie and it containe...,False,AskReddit,https://www.reddit.com/r/AskReddit/comments/dl...,True,self.AskReddit,1
3,Google Shortener is at Goo.gl,False,technology,http://www.imediapress.com/supa/2010/09/30/goo...,False,imediapress.com,0
4,conservative utopia,False,reddit.com,http://politix101.wordpress.com/2010/09/30/wha...,False,politix101.wordpress.com,0
...,...,...,...,...,...,...,...
630293,"You can invite any three people, living or dea...",False,AskReddit,https://www.reddit.com/r/AskReddit/comments/dz...,True,self.AskReddit,3
630294,Symbolism,False,photocritique,http://www.flickr.com/photos/ceballos392/50694...,False,flickr.com,1
630295,Richie Cunning - Last Stop (jazzy and soulful),False,hiphopheads,http://www.youtube.com/watch?v=Fl5OqTRt-Q4,False,youtube.com,6
630296,"Reddit, I have a question",False,AskReddit,https://www.reddit.com/r/AskReddit/comments/dz...,True,self.AskReddit,0


In [96]:
df.shape

(630298, 7)

In [97]:
df.isnull().sum()

title        0
over_18      0
subreddit    0
url          0
is_self      0
domain       0
score        0
dtype: int64

In [98]:
df["subreddit"].value_counts()

reddit.com      172895
pics             34893
AskReddit        26817
politics         22018
funny            16710
                 ...  
nsf                  1
Awesome_Dude         1
midgetsex            1
Drexel               1
twilight             1
Name: subreddit, Length: 5602, dtype: int64

In [99]:
safe_links = df[(df["is_self"] == False) & (df["over_18"] == False)]

In [100]:
safe_counts = safe_links["subreddit"].value_counts()
safe_counts

reddit.com         153181
pics                33185
politics            19649
funny               14601
WTF                 13709
                    ...  
Slayer                  1
Broadway                1
AquaClass               1
SognandoRoma            1
AfricanAmerican         1
Name: subreddit, Length: 4443, dtype: int64

In [101]:
(safe_counts > 256).value_counts()[True]

128

In [102]:
safe_counts[:128]

reddit.com     153181
pics            33185
politics        19649
funny           14601
WTF             13709
                ...  
de                272
socialmedia       263
TrueReddit        261
SciFiScroll       260
skeptic           259
Name: subreddit, Length: 128, dtype: int64

In [103]:
safe_images = safe_links[safe_links["url"].str.match("\.(jpg|png)$")]
safe_images

,title,over_18,subreddit,url,is_self,domain,score


In [104]:
safe_images

,title,over_18,subreddit,url,is_self,domain,score


In [105]:
safe_links["url"].str

In [106]:
safe_links = safe_links.astype(
    {"url": str, "title": str, "domain": str, "title": str, "subreddit": "category"})

In [107]:
safe_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 464238 entries, 0 to 630297
Data columns (total 7 columns):
title        464238 non-null object
over_18      464238 non-null bool
subreddit    464238 non-null category
url          464238 non-null object
is_self      464238 non-null bool
domain       464238 non-null object
score        464238 non-null int64
dtypes: bool(2), category(1), int64(1), object(3)
memory usage: 19.7+ MB


In [108]:
safe_links.dtypes

title          object
over_18          bool
subreddit    category
url            object
is_self          bool
domain         object
score           int64
dtype: object

In [109]:
# endswith did not work, so just using the match function. Also need the .* at the front.
safe_images = safe_links[safe_links["url"].str.match(".*\.(jpg|png)")]
safe_images

,title,over_18,subreddit,url,is_self,domain,score
0,Just when I thought I was getting good !!,False,gaming,http://imgur.com/nWBNd.jpg,False,imgur.com,29
5,This made my day,False,funny,http://i52.tinypic.com/536eo.jpg,False,i52.tinypic.com,0
6,My endless problems with boats.,False,Minecraft,http://imgur.com/Jy3UW.png,False,imgur.com,7
16,Not looking at them...,False,pics,http://thechive.files.wordpress.com/2010/09/be...,False,thechive.files.wordpress.com,409
27,"Boating, you're doing it wrong.",False,pics,http://sphotos.ak.fbcdn.net/hphotos-ak-snc4/hs...,False,sphotos.ak.fbcdn.net,9
...,...,...,...,...,...,...,...
630275,"I'm the Grim Reaper, Lardass, and you're my ne...",False,pics,http://i.imgur.com/uoH4m.jpg,False,i.imgur.com,71
630276,My girlfriend and me. Party on!,False,pics,http://i.imgur.com/w66do.jpg,False,i.imgur.com,0
630284,Am I the only one that forgot about this???,False,reddit.com,http://www.homeandlearn.co.uk/mw/images/wordAr...,False,homeandlearn.co.uk,1
630285,My improvised costume: Business-Casual Ninja,False,pics,http://i.imgur.com/E7lle.jpg,False,i.imgur.com,0


In [110]:
safe_counts = safe_images["subreddit"].value_counts()
safe_counts

pics                   22046
reddit.com              6452
fffffffuuuuuuuuuuuu     5688
funny                   3403
trees                   2196
                       ...  
greenvilleal               0
greenskeptics              0
greece                     0
graphicnovels              0
0x2a_personal              0
Name: subreddit, Length: 4443, dtype: int64

In [111]:
(safe_counts > 256).value_counts()[True]

14

In [112]:
safe_counts[:50]

pics                   22046
reddit.com              6452
fffffffuuuuuuuuuuuu     5688
funny                   3403
trees                   2196
WTF                     2168
Minecraft               2093
gaming                  1350
atheism                  622
politics                 493
itookapicture            461
aww                      440
starcraft                298
tf2                      297
comics                   227
Logos                    221
ColbertRally             199
todayilearned            180
offbeat                  172
geek                     160
Pumpkins                 155
iWallpaper               142
programming              139
Music                    139
humor                    132
sports                   129
technology               122
catpictures              119
apple                    118
food                     116
wallpaper                112
wow                      112
treecomics               103
HighHeels                 98
science       

In [113]:
unsafe = df[df["over_18"] == True]
unsafe

,title,over_18,subreddit,url,is_self,domain,score
14,"Red leather corset, back and [f]ront.",True,gonewild,http://imgur.com/Hq9CF.jpg,False,imgur.com,42
54,japanese tgirl,True,Tgirls,http://www.trannytube.com/media/9888/Japanese_...,False,trannytube.com,10
60,Please give posts from r/treesgonewild the NSF...,True,ideasfortheadmins,https://www.reddit.com/r/ideasfortheadmins/com...,True,self.ideasfortheadmins,5
74,Lexi Belle ENJOY!!,True,nsfw,http://images.4chan.org/gif/src/1285819443182.gif,False,images.4chan.org,1
78,Collateral Damage from Afghanistan Bombing (NS...,True,worldpolitics,http://www.boilingfrogspost.com/2010/09/30/%E2...,False,boilingfrogspost.com,11
...,...,...,...,...,...,...,...
629886,Voluptuous Vampirella [NSFW],True,videos,http://www.break.com/usercontent/2009/2/Volupt...,False,break.com,1
630017,"Yes, these women deserve a shoutout (text nsfw)",True,pics,http://imgur.com/ZBrA9.png,False,imgur.com,1
630133,A little so[m]e thing for the ladies of gw.,True,gonewild,http://imgur.com/ebj7N,False,imgur.com,4
630237,The hottest female muscle clip of all time?,True,hardbodies,http://areaorion.blogspot.com/2010/10/hottest-...,False,areaorion.blogspot.com,0


In [114]:
unsafe_counts = unsafe["subreddit"].value_counts()
unsafe_counts

nsfw            4099
gonewild        2056
WTF              832
pics             774
reddit.com       534
                ... 
TheClubhouse       1
mobileporn         1
Atheism4all        1
JustinSucks        1
PregnantJB         1
Name: subreddit, Length: 636, dtype: int64

In [115]:
# data filtering:
# /r/science moderation challenge -
# https://www.kaggle.com/areeves87/explaining-and-reproducing-the-dataset/data
# used 90th %, but that was for finding removed comments that were on topic and popular.
# I just want to find submissions that are about average. I can find the percentiles

In [116]:
df["score"].describe()

count    630298.000000
mean         11.552139
std          74.964145
min           0.000000
25%           1.000000
50%           1.000000
75%           4.000000
max        3702.000000
Name: score, dtype: float64

In [117]:
# so, 50th percentile are 1, and it takes until the 75th to be 4. I wonder what OpenAI used?
# openai used a threshold of >= 3

In [120]:
good = df[df["score"] >= 3]
good

,title,over_18,subreddit,url,is_self,domain,score
0,Just when I thought I was getting good !!,False,gaming,http://imgur.com/nWBNd.jpg,False,imgur.com,29
1,Someone just sent this to me through my colleg...,False,atheism,https://www.reddit.com/r/atheism/comments/dl97...,True,self.atheism,7
6,My endless problems with boats.,False,Minecraft,http://imgur.com/Jy3UW.png,False,imgur.com,7
9,Looking for a possible costume group.,False,ColbertRally,https://www.reddit.com/r/ColbertRally/comments...,True,self.ColbertRally,6
10,Treasury Reaps Billions as It Sells Citi Shares,False,business,http://dealbook.blogs.nytimes.com/2010/09/30/t...,False,dealbook.blogs.nytimes.com,35
...,...,...,...,...,...,...,...
630282,The ONLY Married News Team,False,pics,http://www.flickr.com/photos/54568034@N06/5133...,False,flickr.com,47
630286,Fucking Melvin,False,treecomics,http://imgur.com/gP2yS.jpg,False,imgur.com,101
630291,"Ok Reddit, if you could leave mankind a one-se...",False,reddit.com,https://www.reddit.com/r/reddit.com/comments/d...,True,self.reddit.com,6
630293,"You can invite any three people, living or dea...",False,AskReddit,https://www.reddit.com/r/AskReddit/comments/dz...,True,self.AskReddit,3


In [119]:
good

,title,over_18,subreddit,url,is_self,domain,score
0,Just when I thought I was getting good !!,False,gaming,http://imgur.com/nWBNd.jpg,False,imgur.com,29
1,Someone just sent this to me through my colleg...,False,atheism,https://www.reddit.com/r/atheism/comments/dl97...,True,self.atheism,7
6,My endless problems with boats.,False,Minecraft,http://imgur.com/Jy3UW.png,False,imgur.com,7
9,Looking for a possible costume group.,False,ColbertRally,https://www.reddit.com/r/ColbertRally/comments...,True,self.ColbertRally,6
10,Treasury Reaps Billions as It Sells Citi Shares,False,business,http://dealbook.blogs.nytimes.com/2010/09/30/t...,False,dealbook.blogs.nytimes.com,35
...,...,...,...,...,...,...,...
630282,The ONLY Married News Team,False,pics,http://www.flickr.com/photos/54568034@N06/5133...,False,flickr.com,47
630286,Fucking Melvin,False,treecomics,http://imgur.com/gP2yS.jpg,False,imgur.com,101
630291,"Ok Reddit, if you could leave mankind a one-se...",False,reddit.com,https://www.reddit.com/r/reddit.com/comments/d...,True,self.reddit.com,6
630293,"You can invite any three people, living or dea...",False,AskReddit,https://www.reddit.com/r/AskReddit/comments/dz...,True,self.AskReddit,3


In [128]:
safe_links = good[(good["is_self"] == False) & (good["over_18"] == False)]
safe_images = safe_links[safe_links["url"].str.match(".*\.(jpg|png)")]
safe_counts = safe_images["subreddit"].value_counts()


In [129]:
(safe_counts > 256).value_counts()[True]

10

In [131]:
safe_counts[:50]

pics                   9977
fffffffuuuuuuuuuuuu    3241
trees                  1516
funny                  1436
reddit.com             1342
Minecraft              1281
WTF                     753
gaming                  519
atheism                 373
aww                     348
itookapicture           247
tf2                     224
politics                175
ColbertRally            152
starcraft               140
Logos                   119
catpictures             107
Pumpkins                106
treecomics               93
wallpaper                87
todayilearned            84
HighHeels                81
wow                      78
comics                   78
GuessTheMovie            72
sports                   66
fnv                      64
lolcats                  64
food                     64
windowshots              60
iWallpaper               57
wallpapers               55
Fitness                  55
doctorwho                53
apple                    52
geek                

In [21]:
# or I can use command line tools like jq. 
"""
cat test_data.txt | jq -ca "if .is_self or .score < 3 then empty else {url, title, score, over_18, is_self} end"
< /mnt/data/datasets/reddit_dumps/RS_v2_2010-10  parallel --progress --pipe -j 10 jq -ca -f filter.jq > out.json

< /mnt/data/datasets/reddit_dumps/RS_2019-08  parallel --progress --pipe -j 10 jq -ca -f filter.jq > out.json
"""

'\ncat test_data.txt | jq -ca "if .is_self or .score < 3 then empty else {url, title, score, over_18, is_self} end"\n< /mnt/data/datasets/reddit_dumps/RS_v2_2010-10  parallel --progress --pipe -j 10 jq -ca -f filter.jq > out.json\n\n< /mnt/data/datasets/reddit_dumps/RS_2019-08  parallel --progress --pipe -j 10 jq -ca -f filter.jq > out.json\n'

In [22]:
filtered = pandas.read_json("out.json", lines=True)

In [23]:
filtered

,score,over_18,title,url,subreddit,domain
0,108,False,Umayyad Mosque - Damascus | 634 AD,https://i.redd.it/gjxlv4y8wkb31.jpg,muslimculture,i.redd.it
1,3168,False,"Floating Feature: Come Rock the Qasaba, and Sh...",https://i.redd.it/pi6z16la7qd31.png,AskHistorians,i.redd.it
2,13,True,#katrinajade love suck Dick,https://i.redd.it/b8ifgynw9qd31.jpg,Katrina_Jade,i.redd.it
3,36,False,Command Wrongly Thinks Antag Survivor Is SCP-106,https://i.redd.it/gci6rjru9qd31.png,SS13,i.redd.it
4,285,True,Another view in my lingerie 💕,https://i.imgur.com/YJEDK4t.jpg,thighhighs,i.imgur.com
...,...,...,...,...,...,...
4689832,1205,False,"My dad, after telling him I didn't want to han...",https://i.redd.it/fsmmv7m5ivj31.jpg,insaneparents,i.redd.it
4689833,71,False,Mya sharing her weekend energy,https://i.redd.it/8c5jnm15ivj31.jpg,akita,i.redd.it
4689834,9,False,Cursed_giraffe,https://i.redd.it/j4aqbsu5ivj31.jpg,cursedcomments,i.redd.it
4689835,40,False,Here's the picture I took of Jason Segel in Ri...,https://i.redd.it/ck7y0m8yhvj31.jpg,philadelphia,i.redd.it


In [24]:
cursed = filtered[filtered["subreddit"] == "cursedimages"]

In [25]:
cursed

,score,over_18,title,url,subreddit,domain
747,10,False,Cursed_shadow,https://i.redd.it/x65rz3bpaqd31.jpg,cursedimages,i.redd.it
1959,367,True,Cursed_roadkill,https://i.redd.it/k3n5hfetcqd31.jpg,cursedimages,i.redd.it
4692,18,False,cursed_door,https://i.redd.it/eu1plsshgqd31.jpg,cursedimages,i.redd.it
4778,11,False,Cursed_Cock,https://i.redd.it/4w0oevzugqd31.jpg,cursedimages,i.redd.it
5102,11,False,Cursed_spiderman,https://i.redd.it/3tl7enrchqd31.jpg,cursedimages,i.redd.it
...,...,...,...,...,...,...
4686213,29,False,Cursed_chad,https://i.redd.it/umku5wqgcvj31.jpg,cursedimages,i.redd.it
4687754,3,False,cursed_xray,https://i.redd.it/8wplsaf7fvj31.jpg,cursedimages,i.redd.it
4688434,7,False,Cursed_kidnap,https://i.redd.it/kik7vcx1gvj31.jpg,cursedimages,i.redd.it
4689233,134,False,Cursed_Piercings,https://i.redd.it/5flnad38hvj31.jpg,cursedimages,i.redd.it


In [26]:
blessed = filtered[filtered["subreddit"] == "blessedimages"]

In [27]:
blessed

,score,over_18,title,url,subreddit,domain
2912,55,False,blessed_energy_drink,https://i.redd.it/vlxnd7b8eqd31.jpg,blessedimages,i.redd.it
3416,26,False,Blessed_Band,https://i.redd.it/h8trufrxeqd31.jpg,blessedimages,i.redd.it
5008,78,False,blessed_masterchief,https://i.redd.it/q9xdrub7hqd31.jpg,blessedimages,i.redd.it
13207,2326,False,Blessed_ride,https://i.redd.it/xewokc0ptqd31.jpg,blessedimages,i.redd.it
13664,106,False,Blessed Shark,https://i.redd.it/e8eximabemd31.jpg,blessedimages,i.redd.it
...,...,...,...,...,...,...
4686003,171,False,Blessed Pizza,https://i.redd.it/p3q2gll4cvj31.jpg,blessedimages,i.redd.it
4686440,68,False,blessed_podcast,https://i.imgur.com/tUJrT7U.jpg,blessedimages,i.imgur.com
4687046,68,False,blessed_passenger,https://i.redd.it/y7ngycwtdvj31.jpg,blessedimages,i.redd.it
4687199,39,False,Blessed Alignment,https://i.redd.it/3cam8f11evj31.png,blessedimages,i.redd.it


In [30]:
blessed.to_json('blessed.json')
cursed.to_json('cursed.json')

In [ ]:
def download_item(item: Item, dir, session):
    skip_types = [".mp4", ".gif"]


    # download file
    parsed_url = urlparse(item.url)

    # test against skipping URLs.


    filename = os.path.basename(parsed_url.path)
    if len(filename) > 20:
        filename = filename[-20:]
    # print("Filename: %s" % (filename,))
    extension = os.path.splitext(filename)[1]
    if extension in skip_types:
        raise ValueError(extension)
    file_path = os.path.join(dir, filename)

    r = requests.get(item.url, stream=True, timeout=5.0)
    # print("Headers: %s" % (r.headers,))
    mime_type = r.headers["Content-Type"]
    mime_info = MIME_TYPES.get(mime_type)
    if not mime_info or mime_info.skip:
        r.close()
        raise ValueError("Skipping Mime type: %s url %s" % (mime_type, item.url,))




    hasher = get_hasher()

    chunk_size = 1024 * 1024

    with open(file_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
            # hash file
            hasher.update(chunk)

    digest = hasher.hexdigest()

    # print(digest)
    # print(base64.b32encode(hasher.digest()))

    # check database to see if the hash exists
    exists = session.query(Item).filter_by(hash=digest).first()
    if not exists:
        # new image, save and move it
        # compute path
        path, new_name = hash_path(digest, 2)
        new_filename = new_name + mime_info.extension

        # move file
        new_path = os.path.join(".", path, new_filename)
        new_dir = os.path.join(dir, path)
        new_location = os.path.join(dir, new_path)

